## Basic Exploration of the 256 kHz Pacific Ocean Audio Data in the AWS Open Data Registry

---
An extensive (5+ years and growing) archive of sound recordings from a deep-sea location [along the eastern margin of the North Pacific Ocean](https://www.mbari.org/at-sea/cabled-observatory/) has been made available through AWS Open data.  Temporal coverage of the recording archive has been 95% since project inception in July 2015.  The original recordings have a sample rate of 256 kHz.  This notebook illustrates basic methods to access and process the original audio data using Python.


 <div style="text-align: center">
 <h3>A delayed version of this data can be heard here on this live audio station</h3>
 <audio controls>
 <source src="http://listen.shoutcast.com/oceansoundscape" type="audio/mpeg">
 Your browser does not support the audio player to play the live stream from Monterey Bay  </audio>
</div>

## Data Overview
The full-resolution audio data are in [WAV](https://en.wikipedia.org/wiki/WAV) format in s3 buckets named <b>pacific-sound-256khz-yyyy</b>, where yyyy is 2015 or later.  Buckets are stored as objects, so the data aren't physically stored in folders or directories as you may be famaliar with, but you can think of it conceptually as follows:

```
pacific-sound-256khz-2021
      |
      individual 10-minute files
```


## Install required dependencies

First, let's install the required software dependencies.



In [ ]:
!pip3 install -q boto3
!pip3 install -q soundfile
!pip3 install -q scipy
!pip3 install -q numpy
!pip3 install -q matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00


### Import all packages

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from six.moves.urllib.request import urlopen
import io
import scipy
from scipy import signal, interpolate
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from sys import prefix
import datetime
import os
import logging as log
import sys
from logging import StreamHandler


## List the contents of a monthly directory
### The recordings are a uploaded in the AWS S3. We use the boto3 to download the files. Since the files are open for public use the id and secret key are not required so the they are left blank.

In [ ]:
s3 = boto3.client('s3',
    aws_access_key_id='',
    aws_secret_access_key='',
    config=Config(signature_version=UNSIGNED))

### In the below box we can adding all the files for the given years in a list. The. files are present in S3 under a bucket and it is given below. You will can select the months in other code block do not change the values for month in the for loop. The hydrophone uploads 6 files for an hour ie each file for 10 minutes.

In [ ]:
year_to_process=2022

#start of month


bucket = f'pacific-sound-256khz-{year_to_process}'


paginator = s3.get_paginator('list_objects_v2')
day_obj_keys = []

for month in range(1,13):
    prefix_day = f'{month:02d}/MARS_{year_to_process}{month:02d}'

    pages = paginator.paginate(Bucket=bucket, Prefix=prefix_day)
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                # print(obj['Key'])
                day_obj_keys.append(obj['Key'])


50602


## Handling missing files
### Each year is supposed to have around 52560 file but many files can be missing so we need to replace the missing days with NaN. This is done in the below code. Here we will create the fake file names for an year and match them with the actual filesname and will the print statements will provide the required information like number of missing files and so on.

In [ ]:



total_count = 0 # This will now represent total expected 10-min slots
n_count = 0     # Missing files (NaNs)
file_count = 0  # Matched files

# total_nums_days is not strictly needed for the matching logic but can be kept for debug print if desired
total_nums_days = 0
monthly_file_keys = []
actual_key_index = 0 # Pointer for the sorted parsed_day_obj_keys

minutes_to_add = 10


parsed_day_obj_keys = []
for key in day_obj_keys:
    try:
        # key format: 'MM/MARS_YYYYMMDD_HHMM
        month_val = int(key[0:2])
        year_val = int(key[8:12])
        day_val = int(key[14:16])
        hour_val = int(key[17:19])
        minute_val = int(key[19:21])

        file_dt = datetime.datetime(year_val, month_val, day_val, hour_val, minute_val)
        parsed_day_obj_keys.append((file_dt, key))
    except (ValueError, IndexError) as e:
        print(f"Warning: Could not parse key '{key}'. Skipping. Error: {e}")
        continue

# Sort the parsed keys by their datetime objects.
parsed_day_obj_keys.sort()


for month in range(1, 13):
  current_date = datetime.datetime(year_to_process, month, 1)

  if month == 12:
        next_month = datetime.datetime(year_to_process + 1, 1, 1)
  else:
        next_month = datetime.datetime(year_to_process, month + 1, 1)

  num_month_days = (next_month - current_date).days
  total_nums_days += num_month_days # Kept for consistency if you use it elsewhere

  for day in range(1, num_month_days + 1):
    for hour in range(0, 24):

      # start_time for the *current hour* (e.g., 2020-01-01 10:00:00)
      start_time = datetime.datetime(year=year_to_process, month=month, day=day, hour=hour, minute=0)

      for chunk_ind in range(0, 6): # Iterate through 6 ten-minute chunks in an hour

        # Calculate the precise start and end time for the current 10-minute slot
        current_slot_start_time = start_time + datetime.timedelta(minutes=chunk_ind * minutes_to_add)
        current_slot_end_time = current_slot_start_time + datetime.timedelta(minutes=minutes_to_add)

        total_count += 1 # Increment total expected slots

        matched_for_this_slot = False

        # find a matching file key from the sorted list
        # This while loop only advances actual_key_index forward
        while actual_key_index < len(parsed_day_obj_keys):
            file_dt, original_key_string = parsed_day_obj_keys[actual_key_index]

            # If the file's timestamp is before the current slot,
            # it means this file was either already processed or is simply too old for this slot.
            # Move to the next file in `parsed_day_obj_keys`.
            if file_dt < current_slot_start_time:
                actual_key_index += 1
                continue # Check the next file key

            # If the file's timestamp falls within the current 10-minute slot:
            # (Using [start, end) interval, typical for time series)
            if current_slot_start_time <= file_dt < current_slot_end_time:
                monthly_file_keys.append(original_key_string)
                file_count += 1
                actual_key_index += 1 # Move to the next file key, as this one has been matched
                matched_for_this_slot = True
                break # Found a match for this slot, stop checking keys for THIS slot

            # If the file's timestamp is after the current slot,
            # then no subsequent files in `parsed_day_obj_keys` (because it's sorted)
            # will match the current slot. So, we stop looking for this slot.
            if file_dt >= current_slot_end_time:
                break # No match for this slot, and current file is too new.

        # If after checking relevant keys, no match was found for the current 10-minute slot
        if not matched_for_this_slot:
            monthly_file_keys.append(np.nan)
            n_count += 1

# --- Final Print Statements ---

print(f'The number of keys present (original day_obj_keys): {len(day_obj_keys)} are present in the year : {year_to_process}')
print(f'Total expected 10-minute slots for the year {year_to_process}: {total_count}')
print(f'Actual number of keys matched: {file_count}')
print(f'Number of missing (NaN) slots: {n_count}')
print(f'The length of the list monthly_file_keys (should equal total_expected_slots): {len(monthly_file_keys)}')

# A consistency check
if file_count + n_count == total_count:
    print("Consistency check: matched files + missing slots = total expected slots. (OK)")
else:
    print("WARNING: Inconsistency detected! Counts do not add up correctly.")

# Check for any remaining unmatched keys in day_obj_keys
unmatched_original_keys = len(day_obj_keys) - file_count
if unmatched_original_keys > 0:
    print(f"Number of original day_obj_keys that were not matched to any slot: {unmatched_original_keys}")
    # To see which ones, you'd need to extract them from parsed_day_obj_keys[actual_key_index:]


The number of keys present (original day_obj_keys): 50602 are present in the year : 2022
Total expected 10-minute slots for the year 2022: 52560
Actual number of keys matched: 50595
Number of missing (NaN) slots: 1965
The length of the list monthly_file_keys (should equal total_expected_slots): 52560
Consistency check: matched files + missing slots = total expected slots. (OK)
Number of original day_obj_keys that were not matched to any slot: 7


### All the blocks till this point need to be run only one time right at the starting. In the below block you can choose the range of day and month.

###month_start= here update the starting day
###mnth_strt_day= here mention the starting month

###end_month= here update the end day
###end_mnth_day= here update the end month

In [ ]:
month_start=12
mnth_strt_day=21

#end of month
end_month=12
end_mnth_day=25

##### Here happens the core processing processes. The current notebook is for pacific sound of 256khz. But we resample it to 48khz. Here we download the actaul files.

##### Resampling: The original audio data is resampled from the original sample rate (orig_sr) to the target sample rate (target_sr).
##### Welch PSD: Uses the scipy.signal.welch method to calculate the Power Spectral Density of each audio segment.
##### Calibration: Sensitivity calibration is applied to the computed PSD using factory calibration data (calfreq and calsens).
##### Data Storage: The results are stored in a .mat file, which is compatible with MATLAB. This includes the calibrated PSD values, frequency bins, and metadata.

##### More explantion about the variables are given on next text block.

In [ ]:

# --- Configuration ---
target_sec = 3600  # 1 hour (3600 seconds)
target_sr = 48000
orig_sr = 256000


output_dir = 'calibrated_psds_yearly/'  # directory for monthly mat files
os.makedirs(output_dir, exist_ok=True)

# Factory calibration data (assuming this is constant)
calfreq = [0, 250, 10000, 20100, 30100, 40200, 50200, 60200, 70300, 80300, 90400, 100400, 110400, 120500, 130500, 140500, 150600, 160600, 170700, 180700, 190700, 200000]
calsens = [-177.90, -177.90, -176.80, -176.35, -177.05, -177.35, -177.30, -178.05, -178.00, -178.40, -178.85, -180.25, -180.50, -179.90, -180.15, -180.20, -180.75, -180.90, -181.45, -181.30, -180.75, -180.30]

f_welch_global = None
isens_truncated_global = None
target_bins = 20000  # band of interest




total_days_in_year = (datetime.date(year_to_process + 1, 1, 1) - datetime.date(year_to_process, 1, 1)).days
total_expected_chunks_in_year = total_days_in_year * 24 * 6

start_date_of_year = datetime.date(year_to_process, 1, 1) # This is the absolute start of monthly_file_keys
start_date_process = datetime.date(year_to_process, month_start, mnth_strt_day) # Start date for this processing run
end_date_process = datetime.date(year_to_process, end_month, end_mnth_day)   # End date for this processing run

current_date = start_date_process # Use this for iterating through days

# --- Second Code Block: Process monthly_file_keys ---
while current_date <= end_date_process:
    print(f"\n--- Processing Date: {current_date.strftime('%Y-%m-%d')} ---")

    absolute_day_number_from_year_start = (current_date - start_date_of_year).days
    day_start_index = absolute_day_number_from_year_start * 144 # 144 chunks per day
    day_end_index = day_start_index + 144

    # Ensure the slice doesn't go out of bounds of monthly_file_keys
    if day_start_index >= len(monthly_file_keys):
        print(f"    Warning: No more data in monthly_file_keys for {current_date.strftime('%Y-%m-%d')} (index {day_start_index}). All hours will be NaN.")
        daily_file_keys_slice = [] # Indicate no keys for this day
    else:
        daily_file_keys_slice = monthly_file_keys[day_start_index:day_end_index]

    hourly_groups_for_day = [daily_file_keys_slice[i:i + 6] for i in range(0, len(daily_file_keys_slice), 6)]

    daily_calibrated_psds = []

    for hour_idx, hour_group in enumerate(hourly_groups_for_day):
        audio_segments_for_hour = [] # This list will hold valid audio arrays
        total_sec_current_hour = 0
        temp_sg_db_columns = []

        print(f"\n  Processing hour group {hour_idx:02d} for {current_date.strftime('%Y-%m-%d')}")

        for key in hour_group:

            if not isinstance(key, str):
                continue
            url = f'https://{bucket}.s3.amazonaws.com/{key}'
            try:
                with urlopen(url) as response:
                    audio_data = response.read()
                x, sr = sf.read(io.BytesIO(audio_data), dtype='float32')

                if sr != orig_sr:
                    print(f"    Warning: Expected {orig_sr} Hz sample rate for {key} but got {sr}. Skipping.")
                    continue
                  # now we have the original sample rate at 'x' and the corresponding scaled voltage values

                new_num_samples = int(len(x) * target_sr / orig_sr)
                v1 = signal.resample(x * 3, new_num_samples)
                v = np.array(v1)

                #since we need to downsample the signal we find the new number of line from the formula given above
                # then we use the resample funtion to produce the new required voltage signals and also by multiplying with 3 we convert the
                #scaled voltage to volts
                #then it is converted to array

                dur = len(v) / target_sr
                total_sec_current_hour += dur
                audio_segments_for_hour.append(v)

                print(f'{key} | duration (s): {dur:.2f} | total_sec so far (this hour): {total_sec_current_hour:.2f}')

                if total_sec_current_hour >= target_sec:
                    break

            except Exception as e:
                print(f"Error processing {key}: {e}. Skipping this file.")
                continue


        if audio_segments_for_hour:
            concatenated_hour_data = np.concatenate(audio_segments_for_hour)

            if concatenated_hour_data.size > 0:
                sample_rate = target_sr
                spa = 3600
                nsec = concatenated_hour_data.size / sample_rate
                nseg = int(nsec / spa)

                if nseg == 0:
                    print(f"    Hour group {hour_idx:02d} has less than {spa} seconds of data. Appending NaN.")
                    daily_calibrated_psds.append(np.full((target_bins,), np.nan))
                    continue

                # nfreq = int(sample_rate / 2 + 1)

                nfft_welch = sample_rate
                nperseg_welch = 2048
                # nperseg_welch = nseg
                noverlap_welch = nperseg_welch // 2 # Fixed to 50% overlap

                segment_length_samples = spa * sample_rate

                for i in range(nseg):
                    start_sample = int(i * segment_length_samples)
                    end_sample = int(start_sample + segment_length_samples)
                    segment = concatenated_hour_data[start_sample:end_sample]

                    current_nperseg = nperseg_welch
                    current_noverlap = noverlap_welch

                    if len(segment) < nperseg_welch:
                        current_nperseg = len(segment)
                        current_noverlap = current_nperseg // 2

                    if current_nperseg < 1:
                        print(f"Warning: Segment {i} in hour group {hour_idx:02d} is too short ({len(segment)} samples). Skipping.")
                        continue

                    f, Pxx = signal.welch(segment, fs=sample_rate, window='hann', nperseg=target_sr, noverlap=target_sr // 2, nfft=nfft_welch)

                    if f_welch_global is None:
                        f_welch_global = f
                        tck = interpolate.splrep(calfreq, calsens, s=0)
                        isens_global = interpolate.splev(f_welch_global, tck, der=0)

                        if f_welch_global.size >= target_bins:
                            isens_truncated_global = isens_global[:target_bins]
                        else:
                            print(f"Warning: f_welch has fewer than {target_bins} bins ({f_welch_global.size}). Calibration will use available bins.")
                            target_bins = f_welch_global.size # Adjust target_bins here
                            isens_truncated_global = isens_global

                    # Always append if Pxx is a valid array of the expected size for the first time or consistent size
                    # Better to check if Pxx is valid rather than specific size match unless strictly required
                    if Pxx.size > 0: # Ensure Pxx is not empty
                        temp_sg_db_columns.append(10 * np.log10(Pxx))
                    else:
                        print(f"Warning: Pxx is empty for segment {i} in hour {hour_idx:02d}. Skipping.")
                        continue

            if len(temp_sg_db_columns) >= 1:
                sg_hour = np.stack(temp_sg_db_columns, axis=1)
                mean_psd_db_hour = np.mean(sg_hour, axis=1)

                print(f"    Successfully computed mean PSD for hour group {hour_idx:02d}")

                if f_welch_global is not None and isens_truncated_global is not None:
                    # Ensure mean_psd_db_hour and isens_truncated_global have compatible shapes
                    if mean_psd_db_hour.shape[0] < target_bins:
                        print(f"    Warning: Hourly PSD has fewer bins than target_bins ({mean_psd_db_hour.shape[0]} vs {target_bins}). Using available bins.")
                        current_mean_psd_db_hour = mean_psd_db_hour
                    else:
                        current_mean_psd_db_hour = mean_psd_db_hour[:target_bins]

                    if isens_truncated_global.shape[0] != current_mean_psd_db_hour.shape[0]:
                        isens_truncated_global_adjusted = isens_truncated_global[:current_mean_psd_db_hour.shape[0]]
                        print(f"    Adjusting isens_truncated_global size from {isens_truncated_global.shape[0]} to {isens_truncated_global_adjusted.shape[0]}.")
                    else:
                        isens_truncated_global_adjusted = isens_truncated_global

                    sg_calibrated = current_mean_psd_db_hour - isens_truncated_global_adjusted
                    daily_calibrated_psds.append(sg_calibrated)
                else:
                    print(f"    Calibration data (f_welch_global or isens_truncated_global) not ready for hour {hour_idx:02d}. Appending NaN vector.")
                    daily_calibrated_psds.append(np.full((target_bins,), np.nan))
            else:
                print(f"    No valid segments could be processed into PSDs for hour group {hour_idx:02d}. Appending NaN vector.")
                daily_calibrated_psds.append(np.full((target_bins,), np.nan))
        else: # This handles cases where concatenated_hour_data is empty
            print(f"    Concatenated hour data is empty for hour {hour_idx:02d}. Appending NaN.")
            daily_calibrated_psds.append(np.full((target_bins,), np.nan))

    # Ensure we have 24 hours (fill with NaN if not)
    while len(daily_calibrated_psds) < 24:
        daily_calibrated_psds.append(np.full((target_bins,), np.nan))

    daily_calibrated_psds_array = np.array(daily_calibrated_psds)

    filename = os.path.join(output_dir, f'calibrated_psd_{current_date.strftime("%Y%m%d")}.mat')


    daily_calibrated_psds_array_for_mat = daily_calibrated_psds_array.T


    mat_data = {
        'calibrated_hourly_PSD_dB': daily_calibrated_psds_array_for_mat,
        'description': (
            "Calibrated hourly Power Spectral Density. "
            f"Matrix columns represent hours (0-23) for {current_date.strftime('%Y-%m-%d')}. "
            "Values are in dB re 1 uPa^2/Hz. "
            "NaNs indicate missing or invalid data for an hour."
        ),
        'frequencies_Hz': f_welch_global[:target_bins] if f_welch_global is not None else np.arange(target_bins),
        'sample_rate_Hz': target_sr,
        'spa_seconds': spa
    }

    scipy.io.savemat(filename, mat_data)
    print(f"--- Saved calibrated PSDs for {current_date.strftime('%Y-%m-%d')} to {filename} ---")

    # Move to the next day
    current_date += datetime.timedelta(days=1)

print("Processing complete.")


--- Processing Date: 2022-12-21 ---

  Processing hour group 00 for 2022-12-21
12/MARS_20221221_000208.wav | duration (s): 600.00 | total_sec so far (this hour): 600.00
12/MARS_20221221_001208.wav | duration (s): 600.00 | total_sec so far (this hour): 1200.00
12/MARS_20221221_002208.wav | duration (s): 600.00 | total_sec so far (this hour): 1800.00
12/MARS_20221221_003208.wav | duration (s): 600.00 | total_sec so far (this hour): 2400.00
12/MARS_20221221_004208.wav | duration (s): 600.00 | total_sec so far (this hour): 3000.00
12/MARS_20221221_005208.wav | duration (s): 600.00 | total_sec so far (this hour): 3600.00
    Successfully computed mean PSD for hour group 00

  Processing hour group 01 for 2022-12-21
12/MARS_20221221_010208.wav | duration (s): 600.00 | total_sec so far (this hour): 600.00
12/MARS_20221221_011208.wav | duration (s): 600.00 | total_sec so far (this hour): 1200.00
12/MARS_20221221_012208.wav | duration (s): 600.00 | total_sec so far (this hour): 1800.00
12/MARS

##### target_sec: The target number of seconds (3600 seconds = 1 hour). This value is used to ensure that each processed hour has a duration of 1 hour.
##### target_sr: The target sample rate (48,000 Hz). The audio data is resampled to this rate.
##### orig_sr: The original sample rate (256,000 Hz). The script checks if the audio data matches this rate.
##### output_dir: Directory to save the calibrated PSD files.
##### calfreq and calsens: Factory calibration frequency points and corresponding sensitivities (in dB).
##### f_welch_global: Placeholder for the frequency array from the Welch PSD calculation.
##### isens_truncated_global: Placeholder for the truncated sensitivity calibration data.
##### target_bins: The number of frequency bins to use in the PSD output (set to 20,000).
##### Date Range Configuration:
##### total_days_in_year: The total number of days in the year to process.
##### total_expected_chunks_in_year: The expected number of chunks based on 24 hours per day and 6 chunks per hour.
##### start_date_of_year: Start date of the year for reference.
##### start_date_process: The actual start date for processing data.
##### end_date_process: The actual end date for processing data.